In [41]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.decomposition import TruncatedSVD
import pickle
from mf_performance_details import mf_performance
import warnings
from flask import jsonify
from flask import request, make_response,Response
warnings.filterwarnings("ignore")

In [2]:
def flatten(t):
        return [item for sublist in t for item in sublist]

In [3]:
def save_obj(obj_data,file_name):
    f = open(file_name,"wb")
    # write the python object (dict) to pickle file
    pickle.dump(obj_data,f)
    # close file
    f.close()

def load_obj(file_name):
    file_to_read = open(file_name, "rb")
    obj_data = pickle.load(file_to_read)
    return obj_data
    

In [42]:
result=get_collaboration_based_funds([148627,148629],"user_mf_data.json")

In [43]:
result.data

b'[{"scheme_name":"SBI Focused Equity Fund","benchmark":"S&P BSE 500 Total Return Index","latest NAV- Regular":"245.9817","latest NAV- Direct":"266.8505","1-Year Return(%)- Regular":"67.58","1-Year Return(%)- Direct":"69.38","3-Year Return(%)- Regular":"26.27","3-Year Return(%)- Direct":"27.57","5-Year Return(%)- Regular":"18.11","5-Year Return(%)- Direct":"19.29"},{"scheme_name":"HDFC Top 100 Fund","benchmark":"NIFTY 100 Total Return Index","latest NAV- Regular":"709.1150","latest NAV- Direct":"749.9720","1-Year Return(%)- Regular":"60.15","1-Year Return(%)- Direct":"61.07","3-Year Return(%)- Regular":"17.75","3-Year Return(%)- Direct":"18.44","5-Year Return(%)- Regular":"13.65","5-Year Return(%)- Direct":"14.39"},{"scheme_name":"LIC MF Bond Fund","benchmark":"CRISIL Composite Bond Index","latest NAV- Regular":"57.7244","latest NAV- Direct":"60.4964","1-Year Return(%)- Regular":"2.27","1-Year Return(%)- Direct":"2.61","3-Year Return(%)- Regular":"7.70","3-Year Return(%)- Direct":"8.33

In [32]:
def get_collaboration_based_funds(current_mf_holdings,user_data_json=None,top_n=5,load_cache=True):
    try:
        CORR_DATA_FILE_NAME="../crosstab_matrix.pkl.bz2"
        COLLAB_MATRIX_FILE_NAME="collaboration_matrix.pkl"
        if load_cache:
            corr_data= pd.read_pickle(CORR_DATA_FILE_NAME)
            collab_matrix=collab_matrix= load_obj(COLLAB_MATRIX_FILE_NAME)
        else:
            user_df=pd.read_json(user_data_json)
            user_df=user_df[["member_id","mf_id"]].drop_duplicates()
            corr_data=correlation_data(user_df)        
            collab_matrix=calculate_collaboration(corr_data)
        suggested_mf= get_suggested_funds(current_mf_holdings,collab_matrix,corr_data)[:top_n]
        result=mf_details(suggested_mf)
        return get_response(result)
    except Exception as ex:
        return get_exception_response("Exception Occured:"+ex) 

In [33]:
def get_suggested_funds(current_mf_holdings,collab_matrix,corr_data):
    list_recomm_mfs=[]
    mf_names = corr_data.columns
    mf_list = list(mf_names)
    for mf in current_mf_holdings:
        mf_idx=mf_list.index(mf)
        mfid=collab_matrix[mf_idx]
        suggested_mf=list(mf_names[(mfid<1.0) & (mfid > 0.9)])
        list_recomm_mfs.append(suggested_mf)
    final_mfs=list(set(flatten(list_recomm_mfs)))
    final_mfs = [e for e in final_mfs if e not in current_mf_holdings]
    return final_mfs

In [34]:
def correlation_data(user_data):
    user_data["buy"]=1
    corr_data=user_data.pivot_table(values="buy",index="member_id",columns="mf_id", fill_value=0)    
    return corr_data

In [35]:
def calculate_collaboration(correlation_data):
    X = correlation_data.T
    ### Decomposing the Matrix
    SVD = TruncatedSVD(n_components=12, random_state=17)
    resultant_matrix = SVD.fit_transform(X)
    ### Generating a Correlation Matrix
    collab_matrix = np.corrcoef(resultant_matrix) 
    return collab_matrix

In [36]:
def mf_details(funds):
    mf_df=pd.read_pickle("mf_data_popular.pkl.bz2",compression="bz2")
    funds_details=mf_df[mf_df["scheme_code"].isin(funds)]
    funds_details=list(set(funds_details["fund_name"].values))
    final_df=get_performance_data(funds_details)
    return final_df

In [37]:
def get_performance_data(fund_names):
    mf_perf=mf_performance()
    df_list=[]
    
    for fund in fund_names:
        df_perf=mf_perf.get_mf_details(fund)
        
        df_list.append(df_perf)
    final_df=pd.concat(df_list).reset_index(drop=True)
    #final_df["risk"]= final_df["scheme_name"].apply(lambda x: mf_perf.get_mf_risk(x))
    return final_df

In [38]:
def get_response(df):
        try:
            json = df.to_json(orient='records')
            return Response(json, mimetype='application/json')
        except Exception as ex:
            return get_exception_response(ex)

In [39]:
def get_exception_response(ex):
    return Response(ex,500)